In [1]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
COLAB_PRE = 'Neuron_Burst_Analysis/'
if RunningInCOLAB:
    !git clone https://github.com/MJC598/Neuron_Burst_Analysis.git
    paths.LOSS_FILE = COLAB_PRE + paths.LOSS_FILE
    paths.PATH = COLAB_PRE + paths.PATH

In [2]:
%matplotlib widget
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
from numpy.random import default_rng
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy.io
from scipy import signal
import random
import time
import pandas as pds
from sklearn.metrics import r2_score, mean_squared_error
import copy

from utils import preprocess, metrics
from config import params, paths
from models import LFPNet

s = 67

rs = RandomState(MT19937(SeedSequence(s)))
rng = default_rng(seed=s)
torch.manual_seed(s)

plt.rcParams.update({'font.size': 16})

In [3]:
plt.close('all')

In [4]:
def train_model(model,save_filepath,training_loader,validation_loader,epochs,device):
    epochs_list = []
    train_loss_list = []
    val_loss_list = []
    training_len = len(training_loader.dataset)
    validation_len = len(validation_loader.dataset)
    
#     feedback_arr = torch.zeros(params.BATCH_SIZE, 90)
    
    #splitting the dataloaders to generalize code
    data_loaders = {"train": training_loader, "val": validation_loader}
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    loss_func = nn.MSELoss()
#     loss_func = nn.L1Loss()
    decay_rate = .99995 #0.98 #decay the lr each step to 98% of previous lr
    lr_sch = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

    total_start = time.time()

    """
    You can easily adjust the number of epochs trained here by changing the number in the range
    """
    for epoch in tqdm(range(epochs), position=0, leave=True):
        start = time.time()
        train_loss = 0.0
        val_loss = 0.0
        temp_loss = 100000000000000.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            for i, (x, y) in enumerate(data_loaders[phase]):
                if params.RECURRENT_NET:
                    x = torch.transpose(x, 2, 1)
                x = x.to(device)
                output = model(x)
                y = y.to(device)
#                 if i%100000 == 0 and epoch%5 == 0:
#                     print(output)
#                     print(y)
                loss = loss_func(torch.squeeze(output), torch.squeeze(y)) 
                #backprop             
                optimizer.zero_grad()
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
#                     if i%100000 == 0 and epoch%5 == 0:
#                         print(model.cn1.weight.grad)
#                         print(model.cn2.weight.grad)
#                         print(model.fc1.weight.grad)
#                         print(model.fc2.weight.grad)

                #calculating total loss
                running_loss += loss.item()
            
            if phase == 'train':
                train_loss = running_loss
                lr_sch.step()
            else:
                val_loss = running_loss

        end = time.time()
        # shows total loss
        if epoch%5 == 0:
            print('[%d, %5d] train loss: %.6f val loss: %.6f' % (epoch + 1, i + 1, train_loss, val_loss))
#         print(end - start)
        
        #saving best model
        if train_loss < temp_loss:
            torch.save(model, save_filepath)
            temp_loss = train_loss
        epochs_list.append(epoch)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
    total_end = time.time()
#     print(total_end - total_start)
    #Creating loss csv
    loss_df = pds.DataFrame(
        {
            'epoch': epochs_list,
            'training loss': train_loss_list,
            'validation loss': val_loss_list
        }
    )
    # Writing loss csv, change path to whatever you want to name it
    
    loss_df.to_csv(paths.LOSS_FILE, index=None)
    return train_loss_list, val_loss_list

In [5]:
f_tr, f_va, filtered = preprocess.get_inVivo_LFP()

(75000, 1, 100) (15000, 1, 100)


In [6]:
# Turn datasets into iterable dataloaders
train_loader = DataLoader(dataset=f_tr,batch_size=params.BATCH_SIZE, shuffle=True)
# tfilt_loader = DataLoader(dataset=t_filt,params.BATCH_SIZE=params.BATCH_SIZE)
val_loader = DataLoader(dataset=f_va,batch_size=params.BATCH_SIZE)
filt_loader = DataLoader(dataset=filtered, batch_size=params.BATCH_SIZE)

In [7]:
model1 = params.MODEL(
    in_size=params.INPUT_SIZE,
    h_size=params.HIDDEN_SIZE,
    out_size=params.OUTPUT_SIZE,
    num_layers=params.NUM_LAYERS,
    dropout=params.DROPOUT
)
model_initial = copy.deepcopy(model1)
model1 = torch.load(paths.PATH)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model1.to(device)

# pnfr_training_loss, pnfr_validation_loss = train_model(model1,paths.PATH,train_loader,
#                                                        val_loader,params.EPOCHS,device)

LFPNetLSTM(
  (activation): LeakyReLU(negative_slope=0.01)
  (dilation_branch): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Conv1d(1, 50, kernel_size=(3,), stride=(1,), dilation=(257,))
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.5, inplace=False)
    (4): Conv1d(50, 50, kernel_size=(3,), stride=(1,), dilation=(129,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Dropout(p=0.5, inplace=False)
    (7): Conv1d(50, 1, kernel_size=(3,), stride=(1,), dilation=(62,))
    (8): LeakyReLU(negative_slope=0.01)
  )
  (convolution_block1): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Conv1d(1, 50, kernel_size=(7,), stride=(2,), padding=(3,))
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.5, inplace=False)
    (4): Conv1d(50, 50, kernel_size=(5,), stride=(1,), padding=(2,))
    (5): LeakyReLU(negative_slope=0.01)
  )
  (convolution_block2): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Conv1d(50, 50, kernel_size=(5,), st

In [8]:
model1 = torch.load(paths.PATH)
model1.eval()

start = 10
k = 100
end= (start + k) if k != None else None

model1.to('cpu')

t_pred, t_real = metrics.r2_eval(model1, train_loader, k=end)
v_pred, v_real, o2, l2 = metrics.r2_eval(model1, val_loader, filt=filt_loader, k=end)

(56320, 100) (56320, 100)
0 steps ahead: -62.24849380120956
1 steps ahead: -0.4201598144620966
2 steps ahead: 0.9239162142980463
3 steps ahead: 0.9091473128203439
4 steps ahead: 0.9262890717800505
5 steps ahead: 0.917984649565689
6 steps ahead: 0.9117589610989203
7 steps ahead: 0.8816378483313783
8 steps ahead: 0.7958236768156848
9 steps ahead: 0.37205618148021147
10 steps ahead: 0.5890361371688875
11 steps ahead: 0.8756708818840553
12 steps ahead: 0.9103817188781008
13 steps ahead: 0.9100619486251283
14 steps ahead: 0.8854785726579845
15 steps ahead: 0.8406013463700406
16 steps ahead: 0.7451695205854243
17 steps ahead: 0.4835815034461871
18 steps ahead: 0.03609568635301674
19 steps ahead: 0.5909746730606497
20 steps ahead: 0.8054954435568056
21 steps ahead: 0.8525908815121941
22 steps ahead: 0.850464551512942
23 steps ahead: 0.8212751286094717
24 steps ahead: 0.7612967475051676
25 steps ahead: 0.6295570008976095
26 steps ahead: 0.34600773052983014
27 steps ahead: 0.07995544350751949
2

In [9]:
print("Train MSE: {:f}".format(mean_squared_error(t_real, t_pred)))
print("Val MSE: {:f}".format(mean_squared_error(v_real, v_pred)))
# print("Full MSE: {:f}".format(mean_squared_error(f_real, f_pred)))
# print("Burst MSE: {:f}".format(mean_squared_error(b_real, b_pred)))
print(t_real.shape)
print(t_pred.shape)

Train MSE: 12899.304688
Val MSE: 13317.976562
(56320, 100)
(56320, 100)


In [10]:
tp, vp, tr, vr = t_pred, v_pred, t_real, v_real

In [11]:
# print(next(iter(train_loader))[0][:,2,0])
# print(next(iter(burst_loader))[0][:,2,0])
# t_pred = tp[:,29]
# v_pred = vp[:,29]
# t_real = tr[:,29]
# v_real = vr[:,29]
t_pred = tp[1,:]
v_pred = vp[1,:]
t_real = tr[1,:]
v_real = vr[1,:]
v_o2 =   o2[1,:]
v_l2 =   l2[1,:]

In [12]:
fig1, ax1 = plt.subplots(nrows=1, ncols=2)
fig1.tight_layout()
ax1[0].plot(range(params.EPOCHS), pnfr_training_loss)
ax1[0].set_title('Training Loss')
ax1[0].set_ylabel('Loss')
ax1[0].set_xlabel('Epoch')

ax1[1].plot(range(params.EPOCHS), pnfr_validation_loss)
ax1[1].set_title('Validation Loss')
ax1[1].set_ylabel('Loss')
ax1[1].set_xlabel('Epoch')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'pnfr_training_loss' is not defined

In [13]:
fig, ax = plt.subplots(nrows=3, ncols=1)
fig.tight_layout()
print(v_real.shape)

end=99

ax[0].plot(np.arange(0,end), t_real[:end], color='blue',label='Labels')
ax[0].scatter(np.arange(0,end), t_pred[:end], color='slateblue',label='Training t+30')
ax[0].set_title('Training LFPNet')
ax[0].set_ylabel('LFP')
ax[0].set_xlabel('Time')

ax[1].plot(np.arange(0,end), v_real[0:end], color='blue',label='Labels')
ax[1].scatter(np.arange(0,end), v_pred[:end], color='slateblue',label='Predicted t+30')
ax[1].set_title('Validation LFPNet')
ax[1].set_ylabel('LFP')
ax[1].set_xlabel('Time')
# ax[2,0].legend()

ax[2].plot(np.arange(0,end), v_l2[:end], color='blue',label='Labels')
ax[2].scatter(np.arange(0,end), v_o2[:end], color='slateblue',label='Training t+30')
ax[2].set_title('Neuronal Component LFPNet')
ax[2].set_ylabel('LFP')
ax[2].set_xlabel('Time')


ax[0].legend(loc=2, prop={'size': 10})

# import plotly.tools as tls
# plotly_fig = tls.mpl_to_plotly(fig)
# plotly_fig.write_html("testfile.html")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(100,)


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
fig.tight_layout()

ax.plot(np.arange(start,end), f_real[start:end], color='blue',label='Labels')
ax.scatter(np.arange(start,end), f_pred[start:end], color='red',label='Training t+10')
ax.set_title('Full LFP vs Time')
ax.set_ylabel('Signal')
ax.set_xlabel('Time')

# ax[1].plot(np.arange(start,end), n_real[start:end], color='blue',label='Labels')
# ax[1].scatter(np.arange(start,end), n_pred[start:end], color='red',label='Training t+10')
# ax[1].set_title('Noise')
# ax[1].set_ylabel('Signal')
# ax[1].set_xlabel('Time')z

# ax.plot(np.arange(start,end), s_real[start:end], color='blue',label='Labels')
# ax.scatter(np.arange(start,end), s_pred[start:end], color='red',label='Training t+10')
# ax.set_title('Sine')
# ax.set_ylabel('LFP')
# ax.set_xlabel('Time')

plt.show()

In [ ]:
import loss_landscapes
import loss_landscapes.metrics
from mpl_toolkits.mplot3d import axes3d, Axes3D 

STEPS = 100
# model_initial = params.MODEL(params.INPUT_SIZE,params.HIDDEN_SIZE,params.OUTPUT_SIZE)
model_final = copy.deepcopy(model1)


# data that the evaluator will use when evaluating loss
x, y = iter(noise_loader).__next__()
metric = loss_landscapes.metrics.Loss(nn.MSELoss(), x, y)


loss_data_fin = loss_landscapes.random_plane(model_final, metric, 10000, STEPS, normalization='model', deepcopy_model=True)
# plt.contour(loss_data_fin, levels=50)
# plt.title('Loss Contours around Trained Model')
# plt.show()

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
X = np.array([[j for j in range(STEPS)] for i in range(STEPS)])
Y = np.array([[i for _ in range(STEPS)] for i in range(STEPS)])
ax.plot_surface(X, Y, loss_data_fin, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
ax.set_title('Surface Plot of Loss Landscape')
fig.show()

In [ ]:
print(model1.state_dict()['conv_block.0.weight'])
print(model1.state_dict()['conv_block.0.bias'])
print(model1.state_dict()['conv_block.2.weight'])
print(model1.state_dict()['conv_block.2.bias'])
print(model1.state_dict()['ck1s1.weight'])
print(model1.state_dict()['ck1s1.bias'])
print(model1.state_dict()['fc1.weight'])
print(model1.state_dict()['fc1.bias'])
print(model1.state_dict()['fc2.weight'])
print(model1.state_dict()['fc2.bias'])

In [ ]:
print(model1.state_dict())